In [ ]:
import os
import path_file
os.chdir(os.path.dirname(os.path.dirname(path_file.__file__)))
os.getcwd()

In [187]:
import sys
import cantera as ct
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

import phases
import feedstock as fs
import fuel as fu
import energy as en
import gasifier as gs

In [188]:
DHfo_liq = phases.Hfo['H2O(l)']*1E-6
DHfo_vap = phases.Hfo['H2O']*1E-6

def Mw(fuel):
    formula = fuel.get_fuel_formula()
    ashMW = fuel.fuelAshMW
    totalMass = 0
    for i, sp in enumerate(['C(gr)', 'H', 'O', 'N', 'S', 'CL']):
        moles = formula[i]
        mw = phases.Mw[sp]
        mass = moles * mw
        totalMass += mass
    totalMass += ashMW * formula[-1]
    return totalMass

def findHHV(fuel, LHV):
    a = fuel.get_fuel_formula()[1]
    M = Mw(fuel)
    return LHV - a / (2*M) * (DHfo_liq - DHfo_vap)

def findLHV(fuel, HHV):
    a = fuel.get_fuel_formula()[1]
    M = Mw(fuel)
    return HHV + a / (2*M) * (DHfo_liq - DHfo_vap)

In [189]:
coalUltimate = [48.42, 3.8, 13.9, 0.89, 0.5, 0]
coalAshWB = 0.335
coalMoistWB = 0.078
coalLHV_WB = 17.4
coalAshDB = coalAshWB / (1 - coalMoistWB)
coalMoistDB = coalMoistWB / (1 - coalMoistWB)
coalLHV_DB = coalLHV_WB / (1 - coalMoistWB)

coalAshComp = [21.2, 0.34, 5.3, 5.74, 0.05, 0.42, 0.12, 0.06, 0, 3.1, 0]

coalMass = 12 # kg
coalDryMass = coalMass * (1 - coalMoistWB)
coal = fs.create_fuel_stream(coalDryMass, coalUltimate, coalAshDB, coalMoistDB, None, coalLHV_DB, coalAshComp)
coal.fuelHHV = findHHV(coal, coal.fuelLHV)

In [190]:
runTime = 170 / 60 # h
airRate = 10.6 # kg/h

air = fs.create_air_from_ER(coal, 0.23)
print(air.get_mass())
# stoichAir = fs.create_air_from_ER(coal, 1.0)
# print(stoichAir.get_mass())
# print()

15.079700160627608


In [191]:
outlet0 = gs.gasify_nonisot(coal, air, T0=298.15, P=101325, heatLossFraction=0.2, charFormation=0.353, directMethaneConv=0.0)
outlet1 = gs.gasify_isot(coal, air, T=591+273.15, P=101325, charFormation=0, directMethaneConv=0)
species = ['H2', 'CO', 'CO2', 'CH4', 'N2']
print(outlet0.T - 273.15)
soma = 0
for i, sp in enumerate(species):
    frac = 100*outlet0.get_syngas_fraction(sp, water=False, nitrogen=True)
    print(f'{sp}: {frac}')
    soma += frac
print(soma)

616.6835472167844
H2: 20.232097616764808
CO: 11.142418163196668
CO2: 14.039191189609296
CH4: 0.7679833942483559
N2: 53.13059864172407
99.31228900554319
